# Deploying Machine Learning Models as API Services With BentoML And AWS Lambda
## Get that model online!
![](images/pexels.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/photo/blue-and-red-galaxy-artwork-1629236/'>Suzy Hazelwood</a>
    </strong>
</figcaption>